In [277]:
import pandas as pd
import matplotlib as plt
import numpy as np

In [278]:
orders = pd.read_csv('./datasets/instacart_orders.csv', sep=';')
products = pd.read_csv('./datasets/products.csv', sep=';')
aisles = pd.read_csv('./datasets/aisles.csv', sep=';')
departments = pd.read_csv('./datasets/departments.csv', sep=';')
order_products = pd.read_csv('./datasets/order_products.csv', sep=';')

## Find and remove duplicate values (and describe why you make your choices)

### `orders` data frame

In [279]:
orders.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 478967 entries, 0 to 478966
Data columns (total 6 columns):
 #   Column                  Non-Null Count   Dtype  
---  ------                  --------------   -----  
 0   order_id                478967 non-null  int64  
 1   user_id                 478967 non-null  int64  
 2   order_number            478967 non-null  int64  
 3   order_dow               478967 non-null  int64  
 4   order_hour_of_day       478967 non-null  int64  
 5   days_since_prior_order  450148 non-null  float64
dtypes: float64(1), int64(5)
memory usage: 21.9 MB


In [280]:
# Check for duplicated orders
orders[orders.duplicated]

,order_id,user_id,order_number,order_dow,order_hour_of_day,days_since_prior_order
145574,794638,50898,24,3,2,2.0
223105,2160484,107525,16,3,2,30.0
230807,1918001,188546,14,3,2,16.0
266232,1782114,106752,1,3,2,NaN
273805,1112182,202304,84,3,2,6.0
284038,2845099,31189,11,3,2,7.0
311713,1021560,53767,3,3,2,9.0
321100,408114,68324,4,3,2,18.0
323900,1919531,191501,32,3,2,7.0
345917,2232988,82565,1,3,2,NaN


In [281]:
# Check for all orders placed Wednesday at 2:00 AM
orders.loc[(orders['order_dow'] == 2) & (orders['order_hour_of_day'] == 2)]

,order_id,user_id,order_number,order_dow,order_hour_of_day,days_since_prior_order
1295,1474941,115569,5,2,2,28.0
2391,1640269,173965,15,2,2,2.0
2550,1223696,55276,11,2,2,30.0
5965,579194,85018,14,2,2,11.0
17680,2004230,203121,4,2,2,8.0
...,...,...,...,...,...,...
457660,1562381,19378,5,2,2,6.0
460316,1986084,117453,81,2,2,8.0
462676,1362113,19654,20,2,2,21.0
472800,1746383,88484,19,2,2,5.0


In [282]:
# Remove duplicate orders
orders.drop_duplicates(inplace=True)

In [283]:
# Double check for duplicate rows
orders[orders.duplicated].shape[0]

0

In [284]:
# Double check for duplicate order IDs only
orders[orders.duplicated(subset='order_id')].shape[0]

0

### `products` data frame

In [285]:
products.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 49694 entries, 0 to 49693
Data columns (total 4 columns):
 #   Column         Non-Null Count  Dtype 
---  ------         --------------  ----- 
 0   product_id     49694 non-null  int64 
 1   product_name   48436 non-null  object
 2   aisle_id       49694 non-null  int64 
 3   department_id  49694 non-null  int64 
dtypes: int64(3), object(1)
memory usage: 1.5+ MB


In [286]:
# Check for fully duplicate rows
products[products.duplicated]

,product_id,product_name,aisle_id,department_id


In [287]:
# Check for just duplicate product IDs
products[products.duplicated(subset='product_id')]

,product_id,product_name,aisle_id,department_id


In [288]:
# Check for just duplicate product names (convert names to lowercase to compare better)
products[products['product_name'].str.upper().duplicated()]

,product_id,product_name,aisle_id,department_id
71,72,NaN,100,21
109,110,NaN,100,21
296,297,NaN,100,21
416,417,NaN,100,21
436,437,NaN,100,21
...,...,...,...,...
49689,49690,HIGH PERFORMANCE ENERGY DRINK,64,7
49690,49691,ORIGINAL PANCAKE & WAFFLE MIX,130,14
49691,49692,ORGANIC INSTANT OATMEAL LIGHT MAPLE BROWN SUGAR,130,14
49692,49693,SPRING WATER BODY WASH,127,11


In [289]:
# Check for duplicate product names that aren't missing
products[(products['product_name'].isna() == False) & (products['product_name'].str.upper().duplicated())]

,product_id,product_name,aisle_id,department_id
2058,2059,Biotin 1000 Mcg,47,11
5455,5456,Green Tea With Ginseng and Honey,96,20
5558,5559,Cream Of Mushroom Soup,69,15
7558,7559,Cinnamon Rolls with Icing,105,13
9037,9038,American Cheese slices,21,16
...,...,...,...,...
49689,49690,HIGH PERFORMANCE ENERGY DRINK,64,7
49690,49691,ORIGINAL PANCAKE & WAFFLE MIX,130,14
49691,49692,ORGANIC INSTANT OATMEAL LIGHT MAPLE BROWN SUGAR,130,14
49692,49693,SPRING WATER BODY WASH,127,11


In [290]:
# Some products have the same product_name but if we look at product_id, all products are different
products[products.duplicated(subset='product_id')].shape[0]

0

### `departments` data frame

In [291]:
departments.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 21 entries, 0 to 20
Data columns (total 2 columns):
 #   Column         Non-Null Count  Dtype 
---  ------         --------------  ----- 
 0   department_id  21 non-null     int64 
 1   department     21 non-null     object
dtypes: int64(1), object(1)
memory usage: 464.0+ bytes


In [292]:
# Departments data frame has only 21 entries so we might as well print 'em all
departments

,department_id,department
0,1,frozen
1,2,other
2,3,bakery
3,4,produce
4,5,alcohol
5,6,international
6,7,beverages
7,8,pets
8,9,dry goods pasta
9,10,bulk


In [293]:
departments[departments.duplicated]

,department_id,department


There are no duplicated or missing data here. The only interesting thing is that department 21 is named 'missing'

In [294]:
# Since departments is a DataFrame with only 2 columns, we can index it by id and make into Series to make working with it faster
departments = departments.set_index('department_id').squeeze()
departments.info()

<class 'pandas.core.series.Series'>
Int64Index: 21 entries, 1 to 21
Series name: department
Non-Null Count  Dtype 
--------------  ----- 
21 non-null     object
dtypes: object(1)
memory usage: 336.0+ bytes


### `aisles` data frame


In [295]:
aisles.info()
aisles.sample(5)

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 134 entries, 0 to 133
Data columns (total 2 columns):
 #   Column    Non-Null Count  Dtype 
---  ------    --------------  ----- 
 0   aisle_id  134 non-null    int64 
 1   aisle     134 non-null    object
dtypes: int64(1), object(1)
memory usage: 2.2+ KB


,aisle_id,aisle
90,91,soy lactosefree
69,70,digestion
18,19,oils vinegars
67,68,bulk grains rice dried goods
20,21,packaged cheese


In [296]:
aisles.duplicated(subset='aisle_id').sum()

0

In [297]:
# Since aisles is a data frame with only 2 coluns one of which is id we can make into a series with an id index
aisles = aisles.set_index('aisle_id').squeeze()
aisles.info()

<class 'pandas.core.series.Series'>
Int64Index: 134 entries, 1 to 134
Series name: aisle
Non-Null Count  Dtype 
--------------  ----- 
134 non-null    object
dtypes: object(1)
memory usage: 2.1+ KB


### `order_products` data frame

In [298]:
order_products.info(null_counts=True)
order_products.head()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 4545007 entries, 0 to 4545006
Data columns (total 4 columns):
 #   Column             Non-Null Count    Dtype  
---  ------             --------------    -----  
 0   order_id           4545007 non-null  int64  
 1   product_id         4545007 non-null  int64  
 2   add_to_cart_order  4544171 non-null  float64
 3   reordered          4545007 non-null  int64  
dtypes: float64(1), int64(3)
memory usage: 138.7 MB


/var/folders/jp/1n4xl2n16154nvh9rxfyrbmc6r6mlp/T/ipykernel_82376/582085705.py:1: FutureWarning: null_counts is deprecated. Use show_counts instead
  order_products.info(null_counts=True)


,order_id,product_id,add_to_cart_order,reordered
0,2141543,11440,17.0,0
1,567889,1560,1.0,1
2,2261212,26683,1.0,1
3,491251,8670,35.0,1
4,2571142,1940,5.0,1


In [299]:
order_products.info(null_counts=True)

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 4545007 entries, 0 to 4545006
Data columns (total 4 columns):
 #   Column             Non-Null Count    Dtype  
---  ------             --------------    -----  
 0   order_id           4545007 non-null  int64  
 1   product_id         4545007 non-null  int64  
 2   add_to_cart_order  4544171 non-null  float64
 3   reordered          4545007 non-null  int64  
dtypes: float64(1), int64(3)
memory usage: 138.7 MB


/var/folders/jp/1n4xl2n16154nvh9rxfyrbmc6r6mlp/T/ipykernel_82376/1849786742.py:1: FutureWarning: null_counts is deprecated. Use show_counts instead
  order_products.info(null_counts=True)


## Find and remove missing values


### `products` data frame


In [300]:
# Are all of the missing product names associated with aisle ID 100?
products_product_name_na = products[products['product_name'].isna()]
print(f'We have a total of {products_product_name_na.shape[0]} missing product names')
products_product_name_na_aisle_100 = products_product_name_na.query('aisle_id == 100')
print(f'Of those {products_product_name_na_aisle_100.shape[0]} are associated with aisle 100')
products_product_name_na_aisle_100['department_id'].value_counts()


We have a total of 1258 missing product names
Of those 1258 are associated with aisle 100


21    1258
Name: department_id, dtype: int64

All products with missing name are in aisle ID 100 in a department with ID 21.

In [301]:
# What is this ailse and department?
print(f"Aisle 100 is {aisles.loc[100]}")
print(f"Department 21 is {departments.loc[21]}")

Aisle 100 is missing
Department 21 is missing


So all of the products with missing names are for some reason in a missing department and in a missing aisle.

In [302]:
# Fill missing product names with 'Unknown'
products['product_name'].fillna('Unknown', inplace=True)
print(f"We now have {products['product_name'].isna().sum()} products with missing name")

We now have 0 products with missing name


### `orders` data frame


We saw in info call above that the only missing values inside orders DataFrame are in days_since_prior_order column.

It's also the only float64 column. Let's check if missing values here correspond to no previous orders and try to convert the column to int64 without losing data.

In [303]:
orders_days_since_prior_order_na = orders[orders['days_since_prior_order'].isna()]
orders_days_since_prior_order_na['order_number'].value_counts()

1    28817
Name: order_number, dtype: int64

We can see that all na values are in orders with no previous orders. We can safely fill na values with 0.

In [304]:
orders['days_since_prior_order'].fillna(0.0, inplace=True)

In [305]:
np.array_equal(orders['days_since_prior_order'], orders['days_since_prior_order'].astype(int))

True

Now we can safely convert 'days_since_prior_order' column to int

In [306]:
orders['days_since_prior_order'] = orders['days_since_prior_order'].astype(int)
orders.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 478952 entries, 0 to 478966
Data columns (total 6 columns):
 #   Column                  Non-Null Count   Dtype
---  ------                  --------------   -----
 0   order_id                478952 non-null  int64
 1   user_id                 478952 non-null  int64
 2   order_number            478952 non-null  int64
 3   order_dow               478952 non-null  int64
 4   order_hour_of_day       478952 non-null  int64
 5   days_since_prior_order  478952 non-null  int64
dtypes: int64(6)
memory usage: 25.6 MB


### `order_products` data frame

In [307]:
# What are the min and max values in this column?
print('Max values\n\n', order_products.max(), sep='')
print('\nMin values\n', order_products.min(), sep='')

Max values

order_id             3421079.0
product_id             49694.0
add_to_cart_order         64.0
reordered                  1.0
dtype: float64

Min values
order_id             4.0
product_id           1.0
add_to_cart_order    1.0
reordered            0.0
dtype: float64


In [323]:
# Save all order IDs with at least one missing value in 'add_to_cart_order'
add_to_card_order_na = order_products[order_products['add_to_cart_order'].isna()]['order_id'].unique()

array([2449164, 1968313, 2926893, 1717990, 1959075,  844733,   61355,
        936852,  264710, 1564093,  129627,  293169, 2849370, 1386261,
       3308010,  903110, 2136777, 3347453, 1888628,  165801, 2094761,
       1038146, 2997021,  813364, 2256933,  171934, 2409109, 1730767,
       1169835,  733526,  404157, 3125735,  747668, 1800005, 1961723,
        871281,  388234, 1648217, 1477139,  102236, 1021563, 1832957,
       2721963,  678116, 1220886, 1673227, 2999801, 1633337, 2470674,
       2625444, 1677118, 2479011, 3383594, 1183255, 1713430, 2652650,
       1598369, 1916118,  854647, 1302315,  888470,  180546, 2621907,
       1308785, 2729254,    9310, 2170451, 2979697, 1625713, 1529171])

In [325]:
# Do all orders with missing values have more than 64 products?
order_products.query('order_id in @add_to_card_order_na')['order_id'].value_counts().min()

65

Yes, all orders with missing values have more than 64 products. I guess in original dataset the type of data was used which prevented counting more than 64 products in one order.

In [326]:
# Replace missing values with 999 and convert column to integer type

order_products['add_to_cart_order'].fillna(999, inplace=True)

np.array_equal(order_products['add_to_cart_order'], order_products['add_to_cart_order'].astype(int))

True

In [328]:
order_products['add_to_cart_order'] = order_products['add_to_cart_order'].astype(int)
order_products.info(null_counts=True)

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 4545007 entries, 0 to 4545006
Data columns (total 4 columns):
 #   Column             Non-Null Count    Dtype
---  ------             --------------    -----
 0   order_id           4545007 non-null  int64
 1   product_id         4545007 non-null  int64
 2   add_to_cart_order  4545007 non-null  int64
 3   reordered          4545007 non-null  int64
dtypes: int64(4)
memory usage: 138.7 MB


/var/folders/jp/1n4xl2n16154nvh9rxfyrbmc6r6mlp/T/ipykernel_82376/3510040024.py:2: FutureWarning: null_counts is deprecated. Use show_counts instead
  order_products.info(null_counts=True)


# [A] Easy (must complete all to pass)

### [A1] Verify that the `'order_hour_of_day'` and `'order_dow'` values in the `orders` tables are sensible (i.e. `'order_hour_of_day'` ranges from 0 to 23 and `'order_dow'` ranges from 0 to 6)

### [A2] What time of day do people shop for groceries?

### [A3] What day of the week do people shop for groceries?

### [A4] How long do people wait until placing another order?


# [B] Medium (must complete all to pass)


### [B1] Is there a difference in `'order_hour_of_day'` distributions on Wednesdays and Saturdays? Plot the histograms for both days and describe the differences that you see.

### [B2] What's the distribution for the number of orders per customer?

### [B3] What are the top 20 popular products (display their id and name)?

# [C] Hard (must complete at least two to pass)


### [C1] How many items do people typically buy in one order? What does the distribution look like?

### [C2] What are the top 20 items that are reordered most frequently (display their names and product IDs)?


### [C3] For each product, what proportion of its orders are reorders?

### [C4] For each customer, what proportion of their products ordered are reorders?

### [C5] What are the top 20 items that people put in their carts first? 